In [8]:
%run -i datapy/analyze/classes.py
%run -i datapy/analyze/descriptives.py
%run -i datapy/analyze/recomendation.py
%run -i datapy/analyze/server.py

In [9]:
igor_path='C:\\Users\\Admin\\Desktop\\strategy_result.csv'
igor_path='C:\\temp\\strategy_full.csv'
anton_path='C:\\Users\\Anton\\repository\\datapy\\data\\strategy_full.csv'
path=anton_path

In [10]:
%run -i datapy/analyze/server.py
s=server()
s.setfile(path,delimiter=';', decimal=',',encoding='1251')

Server initialised
File set


## Определение ролей

In [11]:
# начальное
types_and_roles, missing= s.PredictTypesAndRoles()
missing

,# пропущенных,% пропущенных
ApplicationID,0.0,0.000000
AppFillDate,0.0,0.000000
yr,0.0,0.000000
appl_qtr,0.0,0.000000
DepartmentRegion,0.0,0.000000
LoanProgram,0.0,0.000000
ClientIncomeTotal,2.0,0.006818
CalcCostsLoan,0.0,0.000000
CalcCostsSoc,0.0,0.000000
ClientExpenseTotal,0.0,0.000000


In [12]:
for i in range(len(types_and_roles)):
    print(i, types_and_roles[i].name, types_and_roles[i].type, types_and_roles[i].role)

0 ApplicationID VariableTypeEnum.Scale VariableRoleEnum.REJECTED
1 AppFillDate VariableTypeEnum.Nominal VariableRoleEnum.REJECTED
2 yr VariableTypeEnum.Ordinal VariableRoleEnum.REJECTED
3 appl_qtr VariableTypeEnum.Ordinal VariableRoleEnum.REJECTED
4 DepartmentRegion VariableTypeEnum.Nominal VariableRoleEnum.REJECTED
5 LoanProgram VariableTypeEnum.Nominal VariableRoleEnum.REJECTED
6 ClientIncomeTotal VariableTypeEnum.Scale VariableRoleEnum.REJECTED
7 CalcCostsLoan VariableTypeEnum.Scale VariableRoleEnum.REJECTED
8 CalcCostsSoc VariableTypeEnum.Ordinal VariableRoleEnum.REJECTED
9 ClientExpenseTotal VariableTypeEnum.Scale VariableRoleEnum.REJECTED
10 LoanMonthPayment VariableTypeEnum.Scale VariableRoleEnum.REJECTED
11 LoanAmount VariableTypeEnum.Ordinal VariableRoleEnum.REJECTED
12 CalcClientPTI VariableTypeEnum.Ordinal VariableRoleEnum.REJECTED
13 CalcClientOTI VariableTypeEnum.Ordinal VariableRoleEnum.REJECTED
14 PTI VariableTypeEnum.Ordinal VariableRoleEnum.REJECTED
15 OTI VariableType

In [13]:
#Пользовательская корректировка

types_and_roles[0].role = VariableRoleEnum.ID
types_and_roles[1].role = VariableRoleEnum.DATE
for i in range(2, 21):
    types_and_roles[i].role = VariableRoleEnum.REJECTED
for i in range(22, 35):
    types_and_roles[i].role = VariableRoleEnum.FIXED_RULE
for i in range(35, 39):
    types_and_roles[i].role = VariableRoleEnum.HISTORICAL_RULE
for i in range(39, 41):
    types_and_roles[i].role = VariableRoleEnum.POTENCIAL_RULE


types_and_roles[21].role = VariableRoleEnum.TARGET
types_and_roles[20].role = VariableRoleEnum.NEW_CREDIT
types_and_roles[41].role = VariableRoleEnum.APPROVED
types_and_roles[42].role = VariableRoleEnum.NTU
types_and_roles[43].role = VariableRoleEnum.CREDITED

for i in range(44,len(types_and_roles)): 
    types_and_roles[i].role = VariableRoleEnum.REJECTED

for i in range(48,68): 
    types_and_roles[i].role = VariableRoleEnum.VALUE_COLUMN

types_and_roles_edited=types_and_roles
data_desctiption, data_desctiption_m, one_factor, unique_factor, time = s.AnalyseMarkedData(types_and_roles_edited)
#вывод таблицы с расчетным временем
time

,количество комбинаций,приблизительное время расчета
Total,64,0:00:01.465322


In [14]:
#вывод таблицы со статистикой
data_desctiption

,уровень одобрения (%),уровень NTU (%),дефолтность по одобренным (%),дефолтность по выданным (%)
Total,32.761794,32.129851,2.529548,2.934388


In [15]:
#вывод таблицы со статистикой
data_desctiption_m

,уровень одобрения (%),уровень NTU (%),дефолтность по одобренным (%),дефолтность по выданным (%)
2016-05-31,33.704293,30.896226,1.861042,2.357320
2016-06-30,33.333333,27.892031,2.312925,3.403141
2016-08-31,31.309771,21.912351,2.916667,2.308627
2016-10-31,34.189079,34.088200,1.643489,2.297297
2016-11-30,34.103586,33.294393,1.958384,2.907712
2016-12-31,31.454006,27.358491,3.433476,2.732240
2016-01-31,27.822798,24.660194,2.448980,3.119584
2016-02-29,27.140361,24.034335,2.283105,2.362205
2016-03-31,30.898092,20.783133,3.296703,3.453237
2016-04-30,41.344818,55.913978,2.720115,3.389831


In [16]:
#вывод таблицы с однофакторным анализом
one_factor

,# отказов,% отказов,дефолтность (%),# новых кредитов,% новых кредитов
r_stop_nodata,0.0,0.000000,NaN,0.0,NaN
r_stop_age,197.0,0.671530,13.157895,47.0,23.857868
r_stop_fms,316.0,1.077175,9.541985,43.0,13.607595
r_stop_workingterm,31.0,0.105672,16.666667,2.0,6.451613
r_stop_noincome,166.0,0.565858,19.696970,28.0,16.867470
r_stop_noexpenses,165.0,0.562449,22.727273,31.0,18.787879
r_stop_citizenship,0.0,0.000000,NaN,0.0,NaN
r_internalcredithist,34.0,0.115899,45.161290,6.0,17.647059
r_bki_bad,5915.0,20.162940,32.960098,1161.0,19.628064
r_bki_passable,5185.0,17.674530,8.154788,2523.0,48.659595


In [17]:
#вывод таблицы с анализом уникальных отказов
unique_factor

,# отказов,% отказов от отказов по никальным правилам,% отказов от общего числа,дефолтность (%),# новых кредитов,% новых кредитов
r_stop_nodata,0.0,0.000000,0.000000,NaN,0.0,NaN
r_stop_age,125.0,1.058784,0.426098,12.631579,32.0,25.600000
r_stop_fms,50.0,0.423513,0.170439,0.000000,4.0,8.000000
r_stop_workingterm,12.0,0.101643,0.040905,0.000000,0.0,0.000000
r_stop_noincome,0.0,0.000000,0.000000,NaN,0.0,NaN
r_stop_noexpenses,29.0,0.245638,0.098855,15.384615,8.0,27.586207
r_stop_citizenship,0.0,0.000000,0.000000,NaN,0.0,NaN
r_internalcredithist,29.0,0.245638,0.098855,46.153846,5.0,17.241379
r_bki_bad,2193.0,18.575301,7.475457,23.090909,340.0,15.503876
r_bki_passable,2482.0,21.023209,8.460594,3.917254,1939.0,78.122482


# Пользователь выбирает параметры
после этого должен быть шаг выбора параметров:
опция 1: максимальный возможный уровень одобрения, при уровне риска не выше X% (уровень риска по одобренным или уровень риска по выданным - в зависимости от того, предоставлено ли поле NTU)
опция 2: минимальный уровень риска, при уровне одобрения не ниже Y%

In [18]:
#поиск всех возможных комбинаций
found_combinations =s.FindCombinations()

In [19]:
#поиск комбинаций по параметрам
# 1: максимальный возможный уровень одобрения, при уровне риска не выше X% (уровень риска по одобренным
param_max_dr=15
optimized_by_dr=s.OptimezeRulesDR(param_max_dr)
dr = pd.DataFrame(optimized_by_dr)

# 2: минимальный уровень риска, при уровне одобрения не ниже Y%
param_min_ar=10
optimized_by_ar=s.OptimezeRulesAR(param_min_ar)
ar = pd.DataFrame(optimized_by_ar)

In [20]:
dr

,AR,AR_shifted,DR_aproved,DR_aproved_shifted,DR,DR_shifted
"(r_limit_loanamount, r_limit_loanterm, r_limit_loanburden, r_verification)",56.640305,23.878511,4.525518,1.995970,5.249802,2.315414e+00
"(r_limit_loanamount, r_limit_loanburden, r_verification)",56.599400,23.837606,4.521878,1.992330,5.245579,2.311191e+00
"(r_limit_loanterm, r_limit_loanburden, r_verification)",55.614262,22.852468,4.515344,1.985796,5.238000,2.303611e+00
"(r_limit_loanburden, r_verification)",55.583583,22.821789,4.511024,1.981476,5.232988,2.298600e+00
"(r_limit_loanamount, r_limit_loanterm, r_limit_loanburden, r_verification, r_underwriting)",55.205209,22.443414,4.484000,1.954452,5.201640,2.267251e+00
"(r_limit_loanamount, r_limit_loanburden, r_verification, r_underwriting)",55.167712,22.405918,4.480250,1.950702,5.197289,2.262901e+00
"(r_limit_loanterm, r_limit_loanburden, r_verification, r_underwriting)",54.223480,21.461685,4.469467,1.939918,5.184780,2.250392e+00
"(r_limit_loanburden, r_verification, r_underwriting)",54.192801,21.431006,4.465026,1.935478,5.179629,2.245241e+00
"(r_limit_loanamount, r_limit_loanterm, r_limit_loanburden, r_verification, r_security)",54.100764,21.338969,4.362833,1.833285,5.061080,2.126692e+00
"(r_limit_loanamount, r_limit_loanburden, r_verification, r_security)",54.059858,21.298064,4.358939,1.829391,5.056563,2.122175e+00


# Пользователь вводит своё собственное правило

In [23]:
users_combination=['r_limit_loanamount', 'r_limit_loanburden', 'r_verification', 'r_security']
user_combs=s.FindUsersCombination(users_combination)

In [24]:
user_combs.head(1)

,AR,AR_shifted,DR_aproved,DR_aproved_shifted,DR,DR_shifted
"(r_limit_loanamount, r_limit_loanburden, r_verification, r_security)",54.059858,21.298064,4.358939,1.829391,5.056563,2.122175


# Поиск однофакторных правил

In [25]:
# метод для визуализации найденных правил
def RuleToText(rule):
    column_name=rule[0]
    thr=rule[1]
    trend=rule[2]
    dr_delta=rule[3]
    if trend is None:
        return "Для поля {} не было найдено подходящих правил".format(column_name)
    elif trend:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} выше {}. Что позволит снизить DR на {}%".format(column_name, thr, round(dr_delta*100,2))
    else:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} ниже {}. Что позволит снизить DR на {}%".format(column_name, thr, round(dr_delta*100,2))

In [26]:
#Если пользователь не передал ничего - анализируем всё
full_onefactor_rules=s.RecomendOneFactorRules()

In [17]:
for rule in full_onefactor_rules:
    print(RuleToText(rule))

Рекомендуется ограничить выдачи по тем заявкам, где CalculatedMaxLoanAmount ниже 297987.71875. Что позволит снизить DR на 7.72%
Для поля CalculatedMaxLoanAmountBorrower не было найдено подходящих правил
Рекомендуется ограничить выдачи по тем заявкам, где CalculatedMaxLoanAmountCS ниже 196803.5625. Что позволит снизить DR на 8.33%
Рекомендуется ограничить выдачи по тем заявкам, где CalculatedMaxLoanAmountRUR ниже 297987.71875. Что позволит снизить DR на 7.72%
Рекомендуется ограничить выдачи по тем заявкам, где ClientAge выше 56.5. Что позволит снизить DR на 1.05%
Для поля ClientBankAcounts не было найдено подходящих правил
Для поля ClientCategory не было найдено подходящих правил
Для поля ClientCreditCards не было найдено подходящих правил
Для поля ClientCreditPayment не было найдено подходящих правил
Для поля ClientDependNumber не было найдено подходящих правил
Для поля ClientDeposits не было найдено подходящих правил
Для поля ClientDepositsOthers не было найдено подходящих правил
Для 

In [27]:
#Если пользователь передал набор полей - анализируем по ним
user_columns=['CalculatedMaxLoanAmount','CalculatedMaxLoanAmountCS','CalculatedMaxLoanAmountRUR','ClientJobTerm','ClientCreditPayment']
user_onefactor_rules=s.RecomendOneFactorRules(user_columns)

In [28]:
for rule in user_onefactor_rules:
    print(RuleToText(rule))

Рекомендуется ограничить выдачи по тем заявкам, где CalculatedMaxLoanAmount ниже 297987.71875. Что позволит снизить DR на 7.72%
Рекомендуется ограничить выдачи по тем заявкам, где CalculatedMaxLoanAmountCS ниже 196803.5625. Что позволит снизить DR на 8.33%
Рекомендуется ограничить выдачи по тем заявкам, где CalculatedMaxLoanAmountRUR ниже 297987.71875. Что позволит снизить DR на 7.72%
Рекомендуется ограничить выдачи по тем заявкам, где ClientJobTerm ниже 24.5. Что позволит снизить DR на 1.91%
Для поля ClientCreditPayment не было найдено подходящих правил


# Поиск многофакторных правил

In [29]:
# метод для визуализации найденных мульти-правил
def MultyRuleToText(rule, delta_dr):
    if rule is None:
        return 'Не было найдено подходящих правил. Проверьте, возможно один из столбцов пустой'
    result_text='Рекомендуется ограничить выдачи по заявкам, где '
    for r in rule[:-1]:
        result_text+='{} {} {} и '.format(r[0], r[1], round(r[2],4))
    result_text = result_text[:-3] + '. Что позволит снизить DR на {}%'.format(round(delta_dr*100, 2))
    return result_text

In [30]:
#Если пользователь не передал ничего - анализируем всё
best_multirule, dr_delta=s.RecomendMultyFactorRules()
MultyRuleToText(best_multirule, dr_delta)

'Не было найдено подходящих правил. Проверьте, возможно один из столбцов пустой'

In [31]:
#Если пользователь передал набор полей - анализируем по ним
user_columns=['CalculatedMaxLoanAmount','CalculatedMaxLoanAmountCS','CalculatedMaxLoanAmountRUR','ClientJobTerm','ClientCreditPayment']
user_best_multirule, user_dr_delta=s.RecomendMultyFactorRules(user_columns)
MultyRuleToText(user_best_multirule, user_dr_delta)

datapy/analyze/server.py:76: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  


'Рекомендуется ограничить выдачи по заявкам, где ClientJobTerm <= 25.5. Что позволит снизить DR на 4.96%'